In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install evaluate jiwer torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.8 MB/s eta 0:00:00


In [3]:
import csv
from tqdm import tqdm
import torch
from transformers import pipeline
from datasets import load_dataset
import evaluate

In [4]:
def normalize_text(text: str):
    if text is None:
        return ""
    s = " ".join(text.strip().split())
    return s.lower()

# Load dataset VIVOS
dataset = load_dataset("ademax/vivos-vie-speech2text")
test_dataset = dataset["test"]
print(f"Test split size: {len(test_dataset)} samples")
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

README.md:   0%|          | 0.00/566 [00:00<?, ?B/s]

data/train-00000-of-00004-ded4cd66da578d(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

data/train-00001-of-00004-c9714e3bcd9a48(…):   0%|          | 0.00/404M [00:00<?, ?B/s]

data/train-00002-of-00004-65dd6488b23db6(…):   0%|          | 0.00/413M [00:00<?, ?B/s]

data/train-00003-of-00004-223a6ed1e7afd5(…):   0%|          | 0.00/405M [00:00<?, ?B/s]

data/test-00000-of-00001-dc934696ee90315(…):   0%|          | 0.00/139M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Test split size: 1000 samples


In [5]:
MODEL_WHISPER = "openai/whisper-tiny"
MODEL_PHOWHISPER = "vinai/PhoWhisper-tiny"

device = 0 if torch.cuda.is_available() else -1
print("GPU" if device == 0 else "CPU")

pipe_whisper = pipeline(
    "automatic-speech-recognition",
    model=MODEL_WHISPER,
    chunk_length_s=30,
    device=device
)

pipe_pho = pipeline(
    "automatic-speech-recognition",
    model=MODEL_PHOWHISPER,
    chunk_length_s=30,
    device=device
)

preds_whisper = []
preds_pho = []
refs = []
results = []

GPU


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [6]:
for example in tqdm(test_dataset, desc="Transcribing"):
    audio_input = example["audio"]
    ref_text = normalize_text(example["transcription"])
    refs.append(ref_text)

    try:
        pred1 = pipe_whisper(audio_input)["text"]
    except Exception as e:
        print("Whisper error:", e)
        pred1 = ""

    try:
        pred2 = pipe_pho(audio_input)["text"]
    except Exception as e:
        print("PhoWhisper error:", e)
        pred2 = ""

    pred1_norm = normalize_text(pred1)
    pred2_norm = normalize_text(pred2)

    preds_whisper.append(pred1_norm)
    preds_pho.append(pred2_norm)

    results.append({
        "path": str(example.get("path", "")),
        "ref": ref_text,
        "whisper_pred": pred1_norm,
        "pho_pred": pred2_norm
    })

wer_whisper = wer_metric.compute(predictions=preds_whisper, references=refs)
cer_whisper = cer_metric.compute(predictions=preds_whisper, references=refs)
wer_pho = wer_metric.compute(predictions=preds_pho, references=refs)
cer_pho = cer_metric.compute(predictions=preds_pho, references=refs)

print(f"OpenAI Whisper Tiny     - WER: {wer_whisper:.4f}, CER: {cer_whisper:.4f}")
print(f"VinAI PhoWhisper Tiny   - WER: {wer_pho:.4f}, CER: {cer_pho:.4f}")


Transcribing:   0%|          | 0/1000 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
Transcribing: 100%|██████████| 1000/1000 [09:34<00:00,  1.74it/s]


OpenAI Whisper Tiny     - WER: 0.6214, CER: 0.3933
VinAI PhoWhisper Tiny   - WER: 0.1113, CER: 0.0465


In [7]:
print(test_dataset[0])

{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x795c42951130>, 'transcription': 'Tuy nhiên, ca ghép đã hoàn tất như dự kiến, ông Liêm nói.', 'raw_transcription': 'TUY NHIÊN CA GHÉP ĐÃ HOÀN TẤT NHƯ DỰ KIẾN ÔNG LIÊM NÓI'}
